In [1]:
!pwd

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir("drive/My Drive/textcnn_updated/textcnn")

In [4]:
!ls

clean_data_abstract_ln_200.pkl	linked_diseases_abstract.csv  __pycache__
clean_data.txt			NCBI_corpus		      random.csv
components.py			NCBI_corpus.zip		      random.gsheet
dropout_layers.py		NCBItrainset.csv	      text.txt
first_review.png		network.py		      Untitled.ipynb
glove.42B.300d.txt		predictions-Copy1.ipynb       utils
glove.42B.300d.txt.1		predictions.ipynb


In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11778952399991301129, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9630856536665572298
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13126722347157605191
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11276884378
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2657974374443834829
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
!pip install https://download.pytorch.org/whl/cu91/torch-0.3.1-cp36-cp36m-linux_x86_64.whl

    100% |████████████████████████████████| 531.3MB 30kB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.3.1 which is incompatible.
  Found existing installation: torch 1.0.0
    Uninstalling torch-1.0.0:
      Successfully uninstalled torch-1.0.0


In [0]:
!pip install torchvision

    100% |████████████████████████████████| 2.0MB 12.0MB/s 
fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.3.1 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Input
from keras.layers.embeddings import Embedding

from sklearn.preprocessing import MultiLabelBinarizer
import csv,pandas,re
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from keras.layers import Embedding
import numpy as np
from keras.models import Model

import time,os
import components as com

# Torch imports
import torch
import torch.nn as nn
from torch import optim
import network as net
import traceback
from utils import tensor_utils as tu

from tqdm import tqdm

import pickle
import json
import requests
from pprint import pprint




device = torch.device("cuda")
cpu = torch.device("cpu")

Using TensorFlow backend.


In [0]:
device

In [0]:
use_cuda = True

In [0]:
debug = True
MAX_SEQUENCE_LENGTH = 200


class BadResponseException(Exception):
    pass


class Timer:
    """ Simple block which can be called as a context, to know the time of a block. """
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end = time.perf_counter()
        self.interval = self.end - self.start

In [0]:
#data helpers - copied from https://github.com/bhaveshoswal/CNN-text-classification-keras
def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def _get_class_for_doi_(doi:str) -> list:
    url = "http://id.nlm.nih.gov/mesh/sparql"
    query = """
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
                PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
                PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
                PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
                PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
                PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
                PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
                SELECT DISTINCT ?p ?label ?uri 
                WHERE { mesh:%s meshv:broaderDescriptor* ?uri .
                    ?uri rdfs:label ?p.
                    FILTER NOT EXISTS{
                    ?uri meshv:broaderDescriptor ?x
                    }
                }
                """ % doi
    querystring = {"query":query, 'format':'json'}
    payload = ""
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "c9ace615-1b99-4caf-bcad-97ce1060973e"
        }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    return [x['uri']['value'] for x in response.json()['results']['bindings']]


def get_classes_for_dois(dois:list) -> list:
    cls = []
    for doi in dois:
        cls += _get_class_for_doi_(doi)
    
    return list(set(cls))

In [0]:
def get_terms_of_classes(list_terms) -> list:
    url = "http://id.nlm.nih.gov/mesh/sparql"
    query = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
            PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
            PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
            PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
            PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
            PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
            PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
            SELECT DISTINCT ?class
            WHERE { mesh:%s rdfs:label ?class . }
            """%list_terms
    querystring = {"query":query, 'format':'json'}
    payload = ""
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "c9ace615-1b99-4caf-bcad-97ce1060973e"
        }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    return [x['class']['value'] for x in response.json()['results']['bindings']]

In [0]:
def get_list_for_terms(dois:list) -> list:
  cls = []
  for doi in dois:
    #doi.replace("http://id.nlm.nih.gov/mesh/", "")
    cls += get_terms_of_classes(doi.replace("http://id.nlm.nih.gov/mesh/", ""))
  return list(set(cls))

In [0]:
df = pandas.read_csv('linked_diseases_abstract.csv')
data = [x for x in df.to_records(index=False)] 
def read_till_next(data,index):
    temp = []
    temp_loc = 0
    for d in data[index+1:]:
        if 'a' in d[1] or 't' in d[1]:
            return temp,temp_loc
        else:
            temp_loc = temp_loc + 1
            temp.append([d[2],d[3],d[4]])
#             print([d[2],d[3],d[4]])
#             break
    
    return temp,temp_loc

loc = 0
final_data = []
for index,d in enumerate(data):
    if '|t|' in d[1]:
        continue
    elif loc != 0:
        loc = loc - 1
        continue
    else:
        if '|a|' in d[1]:
            temp_holder = {
                'abstract' : d[1],
                'keys' : []
            }
            keys,loc = read_till_next(data,index)
            temp_holder['keys'] = keys
            final_data.append(temp_holder)
clean_data = []
for f in final_data:
    abstarct = clean_str(f['abstract'].split('|')[2])
    disease = []
    for k in f['keys']:
        disease.append(k[-1])
    clean_data.append((abstarct,list(set(disease))))

In [0]:
#Now chanign the hiearchy of the diseases dataset where we only consider Diseases ontology 
clean_data_upper_class = []
us = []
for c in tqdm(clean_data):
    try:
        labels = []
        for l in c[1]:
            if 'D' == l[0]:
                labels = labels + l.split('|')
        #change the class hiearchy 
        new_labels = list(set(get_classes_for_dois(labels)))
        clean_data_upper_class.append((c[0],new_labels))
    except:
        us.append(c)

clean_data = clean_data_upper_class

100%|██████████| 793/793 [45:00<00:00,  4.44s/it]


In [0]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 22.0MB/s 


In [0]:
with open(f'clean_data_abstract_ln_{MAX_SEQUENCE_LENGTH}.pkl', 'wb') as fp:
  pickle.dump(clean_data, fp)

In [0]:
with open ('clean_data_abstract_ln_200.pkl', 'rb') as fp:
  clean_data_ln_200 = pickle.load(fp)

In [0]:
#creating id map of class that is each label is given a unique id
all_class = list(set([g for f in clean_data for g in f[1]]))
all_class_to_id = {}
for index,value in enumerate(all_class):
    all_class_to_id[value] = index
    

texts = [f[0] for f in clean_data]

#Tokenize,idfy,pad
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [0]:
ids = []
for i in all_class:
    ids.append(i.replace("http://id.nlm.nih.gov/mesh/", ""))

In [0]:
ids

In [0]:
targets = get_list_for_terms(all_class)

In [0]:
targets

['Chemically-Induced Disorders',
 'Physiological Phenomena',
 'Occupational Diseases',
 'Health Occupations',
 'Wounds and Injuries',
 'Bacterial Infections and Mycoses',
 'Diagnosis',
 'Male Urogenital Diseases',
 'Digestive System Diseases',
 'Tissues',
 'Virus Diseases',
 'Biological Phenomena',
 'Reproductive and Urinary Physiological Phenomena',
 'Behavior and Behavior Mechanisms',
 'Behavioral Disciplines and Activities',
 'Respiratory Tract Diseases',
 'Cell Physiological Phenomena',
 'Otorhinolaryngologic Diseases',
 'Immune System Diseases',
 'Musculoskeletal and Neural Physiological Phenomena',
 'Female Urogenital Diseases and Pregnancy Complications',
 'Natural Science Disciplines',
 'Psychological Phenomena',
 'Cardiovascular Diseases',
 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
 'Stomatognathic Diseases',
 'Cells',
 'Nervous System Diseases',
 'Endocrine System Diseases',
 'Pathological Conditions, Signs and Symptoms',
 'Musculoskeletal Diseases',


In [0]:
!pip install scikit-multilearn

    100% |████████████████████████████████| 92kB 3.4MB/s 


In [0]:
#constructing the label matrix 
label_matrix = np.zeros((len(all_class),len(all_class)))
for index,value in enumerate(all_class):
    label_matrix[index][index] = 1
    
#Testing 
label_matrix[all_class_to_id[all_class[1]]]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
def create_multi_hot_label(label_matrix,labels):
    '''
    
        label_matrix --> matrix having one hot vectors
        labels --> [idfy]--> [2,5,6]
        return [0,0,1,0,0,1....]
    '''
    vector = np.zeros((label_matrix.shape[0]))
    for i in labels:
        vector = vector + label_matrix[all_class_to_id[all_class[i]]]
    
    return vector

In [0]:
label_vector = []
for f in clean_data:
    label_id = [all_class_to_id[l] for l in f[1]]
    multi_hot_vector = create_multi_hot_label(label_matrix,label_id)
    label_vector.append(multi_hot_vector)

In [0]:
X = data
Y = label_vector
split_index = int(len(X)*.80)
train_x,val_x = X[:split_index],X[split_index:]
train_y,val_y = Y[:split_index],Y[split_index:]

In [0]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
 

parameters = {'k': range(1,3), 's': [0.5, 0.7, 1.0]}
score = 'f1_macro'

#classifier = GridSearchCV(MLkNN(), parameters, scoring=score)

#classifier = BinaryRelevance(LinearSVC())
#classifier = ClassifierChain(LinearSVC())
#classifier = LabelPowerset(MLPClassifier())
classifier = MLkNN(k=20)
classifier.fit(np.asarray(train_x), np.asarray(train_y))
predictions = classifier.predict(np.asarray(val_x))

accuracy_score(np.asarray(val_y),predictions)

# print (classifier., classifier.best_score_)
# sorted(classifier.cv_results_.keys()

0.025806451612903226

In [0]:
!pip install pycm

In [0]:
from pycm import *

cm = ConfusionMatrix(actual_vector=val_y, predict_vector=predictions)
cm.classes

pycmVectorError: ignored

In [0]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(np.asarray(val_y),predictions)

In [0]:
class BiLstmDot(net.Model):

    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id
        
        self.hiddendim = self.parameter_dict['hidden_size'] * (2 * int(self.parameter_dict['bidirectional']))
        self.number_of_labels = self.parameter_dict['number_of_labels']

        if self.debug:
            print("Init Models")

        self.encoder = com.NotSuchABetterEncoder(
            number_of_layer=self.parameter_dict['number_of_layer'],
            bidirectional=self.parameter_dict['bidirectional'],
            embedding_dim=self.parameter_dict['embedding_dim'],
            max_length = self.parameter_dict['max_length'],
            hidden_dim=self.parameter_dict['hidden_size'],
            vocab_size=self.parameter_dict['vocab_size'],
            dropout=self.parameter_dict['dropout'],
            vectors=self.parameter_dict['vectors'],
            enable_layer_norm=False,
            mode = 'LSTM',
            debug = self.debug).to(self.device)
        
        
        self.dense = com.DenseClf(inputdim=self.hiddendim,        # *2 because we have two things concatinated here
                                  hiddendim=self.hiddendim/2,
                                  outputdim=self.number_of_labels).to(self.device)

    def train(self, data, optimizer, loss_fn, device):
    #
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device)

    def _train_pairwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''

        # Unpacking the data and model from args
        sent_batch, y_label = data['sent_batch'], data['y_label']

        optimizer.zero_grad()

        # Encoding all the data
        hidden = self.encoder.init_hidden(sent_batch.shape[0],self.device)
        _, sent_batch_encoded, _, _ = self.encoder(tu.trim(sent_batch), hidden)

        # Calculating dot score
        pred = self.dense(sent_batch_encoded)


        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
        try:
#             print(torch.max(pred), torch.max(y_label))
            loss = loss_fn(pred, y_label.float())
        except RuntimeError:
            traceback.print_exc()
            print(torch.max(pred.to(cpu)), torch.max(y_label.to(cpu)))
#             print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()
        return loss



    def predict(self, sent, device):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():

            self.encoder.eval()
            self.dense.eval()
            
            hidden = self.encoder.init_hidden(sent.shape[0], self.device)
            _, sent_encoded, _, _ = self.encoder(tu.trim(sent.long()), hidden)
            pred = self.dense(sent_encoded)


            self.encoder.train()
            self.dense.train()
            return pred

    def prepare_save(self):
        """

            This function is called when someone wants to save the underlying models.
            Returns a tuple of key:model pairs which is to be interpreted within save model.

        :return: [(key, model)]
        """
        return [('encoder', self.encoder), ('dense', self.dense)]

    def load_from(self, location):
        # Pull the data from disk
        if self.debug: print("loading Bilstmdot model from", location)
        self.encoder.load_state_dict(torch.load(location)['encoder'])
        if self.debug: print("model loaded with weights ,", self.get_parameter_sum())

In [0]:
#glove_file = "https://s3.eu-central-1.amazonaws.com/maastrichtuniversity-ids-open/metadata/glove.42B.300d.txt" #'glove.42B.300d.txt'

In [0]:
!wget 'https://s3.eu-central-1.amazonaws.com/maastrichtuniversity-ids-open/metadata/glove.42B.300d.txt'

--2019-02-07 11:45:42--  https://s3.eu-central-1.amazonaws.com/maastrichtuniversity-ids-open/metadata/glove.42B.300d.txt
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.64
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025028820 (4.7G) [text/plain]
Saving to: ‘glove.42B.300d.txt.2’

glove.42B.300d.txt. 100%[===================>]   4.68G  11.3MB/s    in 7m 21s  

2019-02-07 11:53:04 (10.9 MB/s) - ‘glove.42B.300d.txt.2’ saved [5025028820/5025028820]



In [0]:
vecs = np.random.randn(len(word_index),300)
embeddings_index = {}
f = open('glove.42B.300d.txt.2', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
vecs = embedding_matrix

In [0]:
parameter_dict = {
    'number_of_layer' : 1,
    'bidirectional' : True,
    'embedding_dim' : 300,
    'max_length' : MAX_SEQUENCE_LENGTH,
    'hidden_size':256,
    'vocab_size':len(word_index),
    'dropout':0.2,
    'vectors':vecs,
    'number_of_labels' : len(all_class)
}

In [0]:
device = torch.cuda.device(0)

In [0]:
loss_fn = nn.BCELoss()
modeler = BiLstmDot( _parameter_dict = parameter_dict,_word_to_id=None,
                                     _device=device,_pointwise=False, _debug=False)

optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, modeler.encoder.parameters()))+
                       list(filter(lambda p: p.requires_grad, modeler.dense.parameters())))

In [0]:
class SimplestSampler:
    """
        Given X and Y matrices (or lists of lists),
            it returns a batch worth of stuff upon __next__
    :return:
    """

    def __init__(self, data, bs: int = 64):

        try:
            assert len(data["x"]) == len(data["y"])
        except AssertionError:

            raise MismatchedDataError(f"Length of x is {len(data['x'])} while of y is {len(data['y'])}")

        self.x = data["x"]
        self.y = data["y"]
        self.n = len(self.x)
        self.bs = bs  # Batch Size

    def __len__(self):
        return self.n // self.bs - (1 if self.n % self.bs else 0)

    def __iter__(self):
        self.i, self.iter = 0, 0
        return self

    def __next__(self):
        if self.i + self.bs >= self.n:
            raise StopIteration

        _x, _y = self.x[self.i:self.i + self.bs], self.y[self.i:self.i + self.bs]
        self.i += self.bs

        return _x, _y

In [0]:
train_data = {
    'x' : train_x,
    'y': np.asarray(train_y)
}

valid_data = {
    'x' : val_x,
    'y' : np.asarray(val_y)
}

train_fact_iter = SimplestSampler(train_data)
valid_fact_iter = SimplestSampler(valid_data)

In [0]:
for dat in train_fact_iter:
    break
dat[0].shape, dat[1].shape

((64, 200), (64, 48))

In [0]:
_x = torch.tensor(dat[0], dtype=torch.long, device=device)
_y = torch.tensor(dat[1], dtype=torch.long, device=device)

data = {
    'sent_batch' : _x,
    'y_label' : _y
}

loss = modeler.train(data, optimizer, loss_fn, device)
op = modeler.predict(_x,device)

In [0]:
loss

tensor(0.6932, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [0]:
def eval(op,_y):
    return torch.mean(op.round()[_y==1])

In [0]:
def simplest_loop(epochs: int,
                  train_data_iter,
                  valid_data_iter,
                  device: torch.device,
                  model,
                  optimizer,
                  loss_fn,
                  eval_fn= None) -> (list, list, list):
    """
        A fn which can be used to train a language model.
        The model doesn't need to be an nn.Module,
            but have an eval (optional), a train and a predict function.
        Data should be a dict like so:
            {"train":{"x":np.arr, "y":np.arr}, "val":{"x":np.arr, "y":np.arr} }
        Train_fn must return both loss and y_pred
        :param epochs: number of epochs to train for
        :param data: a dict having keys train_x, test_x, train_y, test_y
        :param device: torch device to create new tensor from data
        :param opt: optimizer
        :param loss_fn: loss function
        :param train_fn: function to call with x and y
        :param predict_fn: function to call with x (test)
        :param data_fn: a class to which we can pass X and Y, and get an iterator.
        :param eval_fn: (optional) function which when given pred and true, returns acc
        :return: traces
    """

    train_loss = []
    train_acc = []
    val_acc = []
    lrs = []

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = []
        per_epoch_tr_acc = []

        # Train
        with Timer() as timer:

            # Make data

            for x, y in tqdm(train_data_iter):
                optimizer.zero_grad()

                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)
                
                data = {
                    'sent_batch' : _x,
                    'y_label' : _y
                }
                
                loss = model.train(data, optimizer, loss_fn, device)
                y_pred = model.predict(_x,device)

                per_epoch_tr_acc.append(eval_fn(y_pred,_y).item())
                per_epoch_loss.append(loss.item())


        # Val
        with torch.no_grad():

            per_epoch_vl_acc = []
            for x, y in tqdm(valid_data_iter):
                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)

                y_pred = model.predict(_x,device)

                per_epoch_vl_acc.append(eval_fn(y_pred, _y).item())

        # Bookkeep
        train_acc.append(np.mean(per_epoch_tr_acc))
        train_loss.append(np.mean(per_epoch_loss))
        val_acc.append(np.mean(per_epoch_vl_acc))

        print("Epoch: %(epo)03d | Loss: %(loss).5f | Tr_c: %(tracc)0.5f | Vl_c: %(vlacc)0.5f | Time: %(time).3f min"
              % {'epo': e,
                 'loss': float(np.mean(per_epoch_loss)),
                 'tracc': float(np.mean(per_epoch_tr_acc)),
                 'vlacc': float(np.mean(per_epoch_vl_acc)),
                 'time': timer.interval / 60.0})

    return train_acc, train_loss, val_acc

In [0]:
traces = simplest_loop(epochs= 65,
                  train_data_iter=train_fact_iter,
                  valid_data_iter=valid_fact_iter,
                  device=device,
                  model = modeler,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
             eval_fn=eval)

9it [00:01,  7.49it/s]                       
2it [00:00, 30.93it/s]               
 12%|█▎        | 1/8 [00:00<00:00,  7.77it/s]

Epoch: 000 | Loss: 0.46890 | Tr_c: 0.21381 | Vl_c: 0.29936 | Time: 0.022 min


9it [00:01,  7.84it/s]                       
2it [00:00, 30.50it/s]               
 12%|█▎        | 1/8 [00:00<00:00,  7.86it/s]

Epoch: 001 | Loss: 0.23436 | Tr_c: 0.32506 | Vl_c: 0.42406 | Time: 0.019 min


9it [00:01,  7.81it/s]                       
2it [00:00, 29.83it/s]               
 12%|█▎        | 1/8 [00:00<00:00,  7.77it/s]

...

Epoch: 062 | Loss: 0.00193 | Tr_c: 1.00000 | Vl_c: 0.62295 | Time: 0.019 min


9it [00:01,  7.86it/s]                       
2it [00:00, 30.90it/s]               
 12%|█▎        | 1/8 [00:00<00:00,  7.73it/s]

Epoch: 063 | Loss: 0.00178 | Tr_c: 1.00000 | Vl_c: 0.60774 | Time: 0.019 min


9it [00:01,  7.85it/s]                       
2it [00:00, 30.67it/s]               

Epoch: 064 | Loss: 0.00164 | Tr_c: 1.00000 | Vl_c: 0.62656 | Time: 0.019 min


In [0]:
def get_predictions_given_text(abstract):
    global modeler, all_class_to_id, word_index
    
    text_id = np.asarray([word_index.get(word, 0) for word in abstract.split()])
    _x = torch.tensor(text_id, dtype=torch.long, device=device).unsqueeze(0)
    y_pred = modeler.predict(_x, device)
    classes = [all_class[i] for i,x in enumerate(y_pred.round().int().squeeze()) if x.item() == 1]
    terms = get_list_for_terms(classes)
    return terms

In [0]:
val_data =  tokenizer.sequences_to_texts(val_x)

In [0]:
 len(val_data)

155

In [0]:
predictions = [get_predictions_given_text(data_points[index]) for index, data_points in enumerate(val_data)]

RuntimeError: ignored

In [0]:
get_predictions_given_text(val_data[34])

['Neoplasms',
 'Endocrine System Diseases',
 'Male Urogenital Diseases',
 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities',
 'Female Urogenital Diseases and Pregnancy Complications']

In [0]:
val_data[34]

'brca1 suggests that other genetic and non genetic factors play a role in tumourigenesis in these individuals the hras1 variable number of tandem repeats vntr polymorphism located 1 kilobase kb downstream of the hras1 proto oncogene chromosome 11p15 5 is one possible genetic modifier of cancer penetrance individuals who have rare alleles of the vntr have an increased risk of certain types of cancers including breast cancer 2 4 to investigate whether the presence of rare hras1 alleles increases susceptibility to hereditary breast and ovarian cancer we have typed a panel of 307 female brca1 carriers at this locus using a pcr based technique the risk for ovarian cancer was 2 11 times greater for brca1 carriers harbouring one or two rare hras1 alleles compared to carriers with only common alleles p 0 015 the magnitude of the relative risk associated with a rare hras1 allele was not altered by adjusting for the other known risk factors for hereditary ovarian cancer 5 susceptibility to breas